# Vector Stores and Embeddings

## Import Libraries

In [1]:
import os
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import numpy as np
from langchain.vectorstores import Chroma

## Read API Key and Setting Embeddings

In [2]:
api_key = open('../api_key.txt').read()

In [3]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

## Document Loader

In [4]:
loaders = [
    PyPDFLoader('documents/MachineLearning-Lecture01.pdf'),
    PyPDFLoader('documents/MachineLearning-Lecture01.pdf'),
    PyPDFLoader('documents/MachineLearning-Lecture02.pdf'), 
    PyPDFLoader('documents/MachineLearning-Lecture03.pdf')
]

In [5]:
docs = []

for loader in loaders:
    docs.extend(loader.load())

## Document Splitting 

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [7]:
splits = text_splitter.split_documents(docs)

In [8]:
len(splits)

209

## Embeddings

**Lets first start with sum examples**

In [9]:
sentence1 = "I like dogs"
sentence2 = "I like cats"
sentence3 = "The weather is beautiful today"

In [10]:
embedding1 = embeddings.embed_query(sentence1)
embedding2 = embeddings.embed_query(sentence2)
embedding3 = embeddings.embed_query(sentence3)

In [11]:
print(f"Embeddings between Sentence 1 and Sentence 2: {np.dot(embedding1, embedding2)}")
print(f"Embeddings between Sentence 1 and Sentence 3: {np.dot(embedding1, embedding3)}")
print(f"Embeddings between Sentence 2 and Sentence 3: {np.dot(embedding2, embedding3)}")

Embeddings between Sentence 1 and Sentence 2: 0.9171789969688378
Embeddings between Sentence 1 and Sentence 3: 0.788817172916979
Embeddings between Sentence 2 and Sentence 3: 0.776870795213783


## Vector Stores

In [12]:
persist_directory = "db/chroma"

In [13]:
# remove old database files if any
# !rm -rf ./db/chroma

In [14]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [15]:
print(vectordb._collection.count())

209


### Similarity Search

In [16]:
question = "Is there an email I can ask for help"

In [17]:
docs = vectordb.similarity_search(
    query=question,
    k=5
)

In [18]:
len(docs)

5

In [19]:
docs[0].page_content

"cs229-qa@cs.stanford.edu. This goes to an acc ount that's read by all the TAs and me. So \nrather than sending us email individually, if you send email to this account, it will \nactually let us get back to you maximally quickly with answers to your questions.  \nIf you're asking questions about homework probl ems, please say in the subject line which \nassignment and which question the email refers to, since that will also help us to route \nyour question to the appropriate TA or to me  appropriately and get the response back to \nyou quickly.  \nLet's see. Skipping ahead — let's see — for homework, one midterm, one open and term \nproject. Notice on the honor code. So one thi ng that I think will help you to succeed and \ndo well in this class and even help you to enjoy this cla ss more is if you form a study \ngroup.  \nSo start looking around where you' re sitting now or at the end of class today, mingle a \nlittle bit and get to know your classmates. I strongly encourage you to f

**Lets save this so we can use it later**

In [20]:
vectordb.persist()

## Failure Modes

This seems great, and basic similarity search will get you 80% of the way there very easily.

But there are some failure modes that can creep up.

Here are some edge cases that can arise - which being fixed in Retrieval Notebook

### Case 1: Getting Duplicate Chunks

Semantic search fetches all similar documents, but does not enforce diversity.

doc[0] and docs[1] are indentical.

In [21]:
question = "What did they say about Matlab?"

In [22]:
docs = vectordb.similarity_search(
    query=question,
    k=5
)

In [23]:
docs[0]

Document(page_content='those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people call it a free ve rsion of MATLAB, which it sort  of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t s een MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to  learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your  own home computer or something if you \ndon\'t have a MATLAB license, for the purposes of  this class, there\'s also — [inaudible] \nwrite that down [inaudible] MATLAB — there\' s also a software package called Octave \nthat you can download for free off the Internet. And it has somewhat fewer features than MATLAB, but it\'s free, and for the purposes of  this class,

In [24]:
docs[1]

Document(page_content='those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people call it a free ve rsion of MATLAB, which it sort  of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t s een MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to  learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your  own home computer or something if you \ndon\'t have a MATLAB license, for the purposes of  this class, there\'s also — [inaudible] \nwrite that down [inaudible] MATLAB — there\' s also a software package called Octave \nthat you can download for free off the Internet. And it has somewhat fewer features than MATLAB, but it\'s free, and for the purposes of  this class,

### Case 2: Getting Results from Other Documents

The question below asks a question about the third lecture, but includes results from other lectures as well.

In [25]:
question = "what did they say about regression in the third lecture?"

In [26]:
docs = vectordb.similarity_search(
    query=question,
    k=5
)

In [27]:
for doc in docs:
    print(doc.metadata)

{'page': 0, 'source': 'documents/MachineLearning-Lecture03.pdf'}
{'page': 2, 'source': 'documents/MachineLearning-Lecture02.pdf'}
{'page': 14, 'source': 'documents/MachineLearning-Lecture03.pdf'}
{'page': 6, 'source': 'documents/MachineLearning-Lecture03.pdf'}
{'page': 8, 'source': 'documents/MachineLearning-Lecture01.pdf'}


In [28]:
print(docs[4].page_content)

into his office and he said, "Oh, professo r, professor, thank you so much for your 
machine learning class. I learned so much from it. There's this stuff that I learned in your 
class, and I now use every day. And it's help ed me make lots of money, and here's a 
picture of my big house."  
So my friend was very excited. He said, "W ow. That's great. I'm glad to hear this 
machine learning stuff was actually useful. So what was it that you learned? Was it 
logistic regression? Was it the PCA? Was it the data ne tworks? What was it that you 
learned that was so helpful?" And the student said, "Oh, it was the MATLAB."  
So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, 
and we'll actually have a short MATLAB tutori al in one of the discussion sections for 
those of you that don't know it.  
Okay. The very last piece of logistical th ing is the discussion s ections. So discussion 
sections will be taught by the TAs, and atte ndance at discussion secti

Approaches discussed in the **Retrieval Notebook** can be used to address both!